The purpose of this script is to setup MongoDB to store and update information AGM could expect to see on a daily basis. Specifically, we want to simulate data for daily deliveries (which will be visualized in Redis).

Some of the data we may want to simulate include:
- Time of day (Military Time)
- Amount and Type of Inventory on Each Truck
- Charge of the Truck
- Current Objective
- Directions
- ETA
- Time of current delivery
- Delivered? (TOF)

Some of the things we can simulate in scaled real-time:
- Differing traffic levels
- Differing levels of charge
- Dynamic objectives


First we need to plan out what routes our truck will take. Lets start by importing node data

In [ ]:
# Run this to install necessary packages and restart the kernel for osmnx import error to be resolved

# !pip install geopandas
# !pip install folium
# !pip install geopy
# !pip install taxicab
# !pip install osmnx==1.9.1
# !pip install --upgrade osmnx matplotlib numpy

In [1]:
import pymongo
import json
import pprint

import pandas as pd
from IPython.display import display

import warnings

warnings.simplefilter("ignore", category=FutureWarning)

import geopandas as gpd
import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium.features import DivIcon
import requests 
import numpy as np
import random
from geopy.geocoders import Nominatim
from IPython.display import Image, IFrame 
from IPython.core.display import HTML 
from pandas import json_normalize
%matplotlib inline
import osmnx as ox
from osmnx import plot_route_folium
import networkx as nx
import matplotlib
import matplotlib as plt
import taxicab as tc
from IPython.display import clear_output
import csv

/opt/conda/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
locations_df = pd.read_csv("locations_project.csv")

locations_df.head()

name     county  \
0                               Fr Dental Laboratory  San Mateo   
1                         Pacifica Spindrift Players  San Mateo   
2                   Pulgas Ridge Open Space Preserve  San Mateo   
3                                              GoPro  San Mateo   
4  Forensic Laboratory of San Mateo Co. Sheriff's...  San Mateo   

                coordinates  esti. pop.          nearest node one  \
0  (37.485814, -122.238715)        2651  (37.478609, -122.291883)   
1  (37.592423, -122.489007)         306  (37.459552, -122.429672)   
2  (37.478609, -122.291883)        1745  (37.485814, -122.238715)   
3  (37.534532, -122.331311)         494   (37.513253, -122.33258)   
4   (37.513253, -122.33258)        2878  (37.534532, -122.331311)   

             nearest node 2            nearest node 3    distance 1  \
0  (37.444031, -122.161216)  (37.491431, -122.228803)   5629.155382   
1  (37.462736, -122.429145)  (37.633669, -122.397713)  21589.095445   
2   (37.513253, -122.33258)  (37.491431, -122.228803)   5606.499382   
3  (37.535644, -122.334809)  (37.545193, -122.306168)   4426.607702   
4  (37.535644, -122.334809)  (37.545193, -122.306168)   4426.607702   

     distance 2    distance 3  
0   7902.135854   1246.209503  
1  21269.789077  15705.573372  
2   8101.156342   6747.760637  
3   2830.915494   5630.481052  
4   3428.965861   5643.734418

In [3]:
set(locations_df['county'])

{'Alameda', 'Contra Costa', 'San Francisco', 'San Mateo', 'Santa Clara'}

In [4]:
# Define the order of nodes to be traversed in each county (Based off of Betweeness)

order = {}

for county in set(locations_df['county']):
    
    order[county] = pd.read_csv(f"{county} Betweeness Results", delimiter='\t')
    order[county] = order[county].drop('Unnamed: 0', axis=1)
    
#     print(order[county].head())

In [5]:
maps = {}

for county in set(locations_df['county']):
    maps[county] = ox.graph_from_place(f'{county}, California, USA', network_type='drive')

In [6]:
# Define the routes we want to take in each county

county_routes = {}

for k, county in enumerate(set(locations_df['county'])):
    
    temp_route_mat = []
    
    for i in range(order[county].shape[0]):
        
        orig = order[county].iloc[i]['coordinates']
        
        temp_route_arr = []
        
        for j in range(order[county].shape[0]):

            dest = order[county].iloc[j]['coordinates']
            
            if orig != dest:
                
                lat_orig, lng_orig = orig.strip("()").split(", ")
                lat_dest, lng_dest = dest.strip("()").split(", ")
                
                try:
                    origin_node = ox.distance.nearest_nodes(maps[county], X=float(lng_orig), Y=float(lat_orig))
                    destination_node = ox.distance.nearest_nodes(maps[county], X=float(lng_dest), Y=float(lat_dest))

                    route = nx.shortest_path(maps[county], origin_node, destination_node)
                    
                    temp_route_arr.append(route)
                
                except:
                    temp_route_arr.append(-1)                
                
                clear_output(wait=True)
                print(f"{county} | {k+1}/{len(set(locations_df['county']))} counties | Node {(i+1)} {(j+1)} Paths Covered")

        temp_route_mat.append(temp_route_arr)
        
        
    county_routes[county] = temp_route_mat
    

Santa Clara | 5/5 counties | Node 20 19 Paths Covered


In [7]:
for k, county in enumerate(set(locations_df['county'])):
    print(county)
    for route in county_routes[county]:
        try:
            print(route.index(-1))
        except:
            pass

San Francisco
Contra Costa
San Mateo
0
Alameda
Santa Clara


In [6]:
# Route Order of each County

county_order = {}

for county in set(locations_df['county']):
    
    county_order[county] = pd.read_csv(f'{county} Betweeness Results', delimiter='\t').drop('Unnamed: 0',axis=1)
    

In [7]:
# Remove any nodes from the path which don't match

county_order['San Mateo'] = county_order['San Mateo'][3::]


In [8]:
county_routes = {}
county_coordinates = {}

for county in set(locations_df['county']):
    
    temp_route_arr = []
    temp_route_coordinates = []
    
    for j in range(county_order[county].shape[0]):
        temp_route_coordinates.append(county_order[county].iloc[j]['coordinates'])
        
    county_coordinates[county] = temp_route_coordinates
    
    for i in range(county_order[county].shape[0] - 1):
        
        orig = county_order[county].iloc[i]['coordinates']
        dest = county_order[county].iloc[i+1]['coordinates']

        lat_orig, lng_orig = orig.strip("()").split(", ")
        lat_dest, lng_dest = dest.strip("()").split(", ")

        origin_node = ox.distance.nearest_nodes(maps[county], X=float(lng_orig), Y=float(lat_orig))
        destination_node = ox.distance.nearest_nodes(maps[county], X=float(lng_dest), Y=float(lat_dest))

        route = nx.shortest_path(maps[county], origin_node, destination_node)

        temp_route_arr.append(route)

        clear_output(wait=True)
        print(f"{county} | Node {(i+1)} {(j+1)} Paths Covered")
        
    county_routes[county] = temp_route_arr


Santa Clara | Node 19 20 Paths Covered


In [9]:
for county in set(locations_df['county']):
    print(len(county_routes[county]))

19
19
19
16
19


Next lets set up our MongoDB:

In [10]:
mongo = pymongo.MongoClient("mongodb://mongo:27017/")

In [11]:
mongo.drop_database("AGM")

In [12]:
db_list = mongo.database_names()

db_list

/tmp/ipykernel_172/3002110245.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  db_list = mongo.database_names()


['admin', 'config', 'local']

So now that we've connected to MongoDB and initialized the database, its time to think about what we'd want to store on MongoDB. Generally, we'd like to store information about routes which won't need to be updated in real time too often.

Some ideas for information to store ahead of time would be:
- Routes we need to take between nodes of each county
- A changing amount of daily requests for food of each store (by product type)
- Order fulfilled tag

After a delivery is made we can update:
- Order fulfiled tag
- Time of Arrival

So lets start by initializing a new database

In [13]:
my_client = mongo["AGM"]

We might want to split the routes and the store info into two separate dbs

In [14]:
db_routes = my_client["AGM_routes"]
db_store_info = my_client["AGM_store_info"]

MongoDB only accepts and returns JSON format data. Therefore, its time to generate the JSON files for out routes and store information.

In [15]:
list(county_order['San Mateo']['name'])

['CourseHero',
 'Studio2325',
 'RoyalBloom',
 'IntelligentLearningPreschool',
 'TeslaMotorsPaloAlto',
 'TheIntelExperienceatBestBuyMountainView',
 'CollegeofSanMateo',
 'FrDentalLaboratory',
 'UnitedAirlinesSanFranciscoMaintenanceCenter',
 'ParkSchool',
 'BloomSkincare',
 'UrbanSanctuary',
 'PacificaSpindriftPlayers',
 'GoPro',
 'RaulCastilloMartialArts',
 'StanfordLawSchoolWilliamH_NeukomBuilding',
 'LakeviewMontessori']

In [16]:
# JSON for routes
 
routes_dictionary = {}

for county in county_order.keys():
    
    info = {}
    
    info['nodes'] = list(county_order[county]['name'])
    info['coordinates'] = county_coordinates[county]
    info['route'] = county_routes[county]
    
    routes_dictionary[county] = info
    
    
routes_json_object = json.dumps(routes_dictionary, indent=4)
 
# Writing to sample.json
with open("county_routes.json", "w") as outfile:
    outfile.write(routes_json_object)

In [17]:
# JSON for stores

rng = np.random.default_rng()

stores_dictionary = {}


for county in county_order.keys():
    
    info = {}
    
    for name in list(county_order[county]['name']):
        
        info['name'] = name
        info['items_requested'] = {"Teriyaki Chicken":int(rng.integers(10, 100)), 
                                   "Eggplant Lasagna":int(rng.integers(10, 100)), 
                                   "Curry Chicken":int(rng.integers(10, 100)), 
                                   "Brocolli Stir Fry":int(rng.integers(10, 100)), 
                                   "Pistachio Salmon":int(rng.integers(10, 100)), 
                                   "Spinach Orzo":int(rng.integers(10, 100)), 
                                   "Chicken Salad":int(rng.integers(10, 100)), 
                                   "Tilapia Piccata":int(rng.integers(10, 100))} 
        info['delivered_tag'] = False
        info['time of delivery'] = "00:00:00"
        
        stores_dictionary[name] = info
    
stores_json_object = json.dumps(stores_dictionary, indent=4)
 
# Writing to sample.json
with open("store_info.json", "w") as outfile:
    outfile.write(stores_json_object)

Now that we have our JSON objects, we need to add them to our MongoDB

In [18]:
# db_routes = my_client["AGM_routes"]
# db_store_info = my_client["AGM_store_info"]

db_routes.insert_one(routes_dictionary)
db_store_info.insert_one(stores_dictionary)


In [19]:
# Find all documents in the collection
cursor = db_routes.find()

# Iterate over the cursor and print each document
for document in cursor:
    print(document)

{'_id': ObjectId('65fd00e395522cd3cb5a2359'), 'Alameda': {'nodes': ['PhantomMotors', 'BusStop5077328SanLeandroBart', 'CaliforniaExpressMotors', 'HaywardBARTGarage', 'MensWearhouse', 'RedApplePizza', 'Up2SpeedSportsTherapy', 'SimbolMaterials', 'Logitech', 'MyUnionCityPlumberHero', 'SpeeDeeOilChangeandTuneUp', 'EnvieInteractive', 'UniversityUnion', 'SpeedeeOilChangeTuneUp', 'SafeAmericaCreditUnion', 'TintNSound', 'DesignEnergyGroup', 'RandallE_Strauss', 'EmcorIntegratedSolutions', 'PhoPlay'], 'coordinates': ['(37.678241, -122.09081)', '(37.695409, -122.077991)', '(37.656344, -122.071653)', '(37.670639, -122.087372)', '(37.692118, -121.927759)', '(37.696829, -122.139635)', '(37.698831, -121.917708)', '(37.661614, -121.903984)', '(37.552155, -122.063208)', '(37.581319, -122.017075)', '(37.708072, -121.924819)', '(37.411732, -122.014626)', '(37.654885, -122.055788)', '(37.693297, -122.066176)', '(37.695814, -121.932341)', '(37.769199, -122.238094)', '(37.698278, -121.919474)', '(37.807695, 

In [20]:
# Find all documents in the collection
cursor = db_store_info.find()

# Iterate over the cursor and print each document
for document in cursor:
    print(document)

{'_id': ObjectId('65fd00e395522cd3cb5a235a'), 'PhantomMotors': {'name': 'PhoPlay', 'items_requested': {'Teriyaki Chicken': 23, 'Eggplant Lasagna': 83, 'Curry Chicken': 16, 'Brocolli Stir Fry': 53, 'Pistachio Salmon': 95, 'Spinach Orzo': 59, 'Chicken Salad': 65, 'Tilapia Piccata': 87}, 'delivered_tag': False, 'time of delivery': '00:00:00'}, 'BusStop5077328SanLeandroBart': {'name': 'PhoPlay', 'items_requested': {'Teriyaki Chicken': 23, 'Eggplant Lasagna': 83, 'Curry Chicken': 16, 'Brocolli Stir Fry': 53, 'Pistachio Salmon': 95, 'Spinach Orzo': 59, 'Chicken Salad': 65, 'Tilapia Piccata': 87}, 'delivered_tag': False, 'time of delivery': '00:00:00'}, 'CaliforniaExpressMotors': {'name': 'PhoPlay', 'items_requested': {'Teriyaki Chicken': 23, 'Eggplant Lasagna': 83, 'Curry Chicken': 16, 'Brocolli Stir Fry': 53, 'Pistachio Salmon': 95, 'Spinach Orzo': 59, 'Chicken Salad': 65, 'Tilapia Piccata': 87}, 'delivered_tag': False, 'time of delivery': '00:00:00'}, 'HaywardBARTGarage': {'name': 'PhoPlay

In [21]:
db_list = mongo.database_names()

db_list

/tmp/ipykernel_172/3002110245.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  db_list = mongo.database_names()


['AGM', 'admin', 'config', 'local']

Our AGM db is ready to go!